## Chap05-2. TensorBoard
### 텐서보드를 이용하기 위해 각종 변수들을 설정하고 저장하는 방법 익히기

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# 1) 학습데이터 불러오기

data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32', skiprows=0)

# 털, 날개, 기타, 포유류, 조류
#  - x_data = 0, 1 (털, 날개)
#  - y_data = 2, 3, 4 (기타, 포유류, 조류)
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [3]:
# 2) 신경망 모델 구성
# glbal_step: 학습 횟수 카운트 변수 설정
#  - 학습에 직접 사용되지 않고, 학습 횟수를 카운트하는 변수
global_step = tf.Variable(0, trainable=False, name='global_step')

# Input(X), Output(Y) 설정
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# input -> hidden(1)[10] -> hidden(2)[20] -> output
# Parameter 설정: W만 설정 b는 생략

# tf.name_scope()를 이용해 텐서보드에서 한 계층 내부를 표현해줌
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    tf.summary.histogram("Weights", W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)


with tf.name_scope('optimizer'):
    # Loss Function: Cross-Entropy
    cost = tf.losses.softmax_cross_entropy(onehot_labels=Y, logits=model)

    # Optimizer: Adam
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

    # global_step으로 넘겨준 변수를, 파라미터(W1, W2, W3)들을 최적화 할 때 마다 학습 획수를 하나씩 증가
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    # tf.summary.scalar를 이용해 손실값을 추적하기 위해 수집할 값을 지정
    tf.summary.scalar('cost', cost)
    
    
# 모델을 저장하고 불러오는 API를 초기화
#  - tf.global_variables() : 앞에서 정의한 변수들(tf.Variable) 가져오는 함수
#                            이 변수들을 파일에 저장하거나 불러올 변수들로 사용함
saver = tf.train.Saver(tf.global_variables())

In [5]:
# 3) 신경망 모델 학습
sess = tf.Session()

# ./model 디렉터리에 기존에 학습해둔 모델이 있는지 확인 후
# - 모델이 있다면 saver.restore()를 사용해 학습된 값들을 불러오고,
# - 모델이 없다면 변수를 새로 초기화함
# - 학습된 모델을 저장한 파일을 체크포인트 파일(checkpoint file)이라고 함
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    

# 텐서보드에서 표시해주기 위한 데이터들을 수집
merged = tf.summary.merge_all()
# 그래프와 텐서값들을 저장할 디렉터리를 설정
writer = tf.summary.FileWriter('./logs', sess.graph)
# logs 디렉터리에 저장된 로그는, 학습 후 
# tensorboard --logdir=./logs 명령어를 이용해 
# http://localhost:6006 에서 확인할 수 있음

# 최적화 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: {:d}  Cost: {:.3f}'.format(sess.run(global_step), 
                                           sess.run(cost, feed_dict={X: x_data, Y: y_data})))
    
    # 적절한 시점에 저장할 값들을 수집하고 저장
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step)) # global_step을 이용해 시점을 기록
    
# 최적화가 끝난 뒤, 변수를 저장
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

INFO:tensorflow:Restoring parameters from ./model\dnn.ckpt-200
Step: 201  Cost: 0.550
Step: 202  Cost: 0.550
Step: 203  Cost: 0.550
Step: 204  Cost: 0.550
Step: 205  Cost: 0.550
Step: 206  Cost: 0.550
Step: 207  Cost: 0.550
Step: 208  Cost: 0.550
Step: 209  Cost: 0.550
Step: 210  Cost: 0.550
Step: 211  Cost: 0.550
Step: 212  Cost: 0.550
Step: 213  Cost: 0.550
Step: 214  Cost: 0.550
Step: 215  Cost: 0.550
Step: 216  Cost: 0.550
Step: 217  Cost: 0.550
Step: 218  Cost: 0.550
Step: 219  Cost: 0.550
Step: 220  Cost: 0.550
Step: 221  Cost: 0.550
Step: 222  Cost: 0.550
Step: 223  Cost: 0.550
Step: 224  Cost: 0.550
Step: 225  Cost: 0.550
Step: 226  Cost: 0.550
Step: 227  Cost: 0.550
Step: 228  Cost: 0.550
Step: 229  Cost: 0.550
Step: 230  Cost: 0.550
Step: 231  Cost: 0.550
Step: 232  Cost: 0.550
Step: 233  Cost: 0.550
Step: 234  Cost: 0.550
Step: 235  Cost: 0.550
Step: 236  Cost: 0.550
Step: 237  Cost: 0.550
Step: 238  Cost: 0.550
Step: 239  Cost: 0.550
Step: 240  Cost: 0.550
Step: 241  Cost: 

'./model/dnn.ckpt-300'

In [5]:
# 4) 결과 확인
#  - 0:기타, 1: 포유류, 2: 조류

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


### TensorBoard in Jupyter Notebook

In [6]:
# TensorFlow Graph visualizer code
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [7]:
show_graph(tf.get_default_graph().as_graph_def())

In [10]:
tf.VERSION

'1.4.0'